In [58]:
import os
from dotenv import load_dotenv
import requests
import csv
import time

In [5]:
def get_bearer_token():
    load_dotenv()
    return os.getenv('TWITTER_BEARER_TOKEN')

In [7]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [65]:
def create_url(user_id, max_results = 10):
    
    search_url = f"https://api.twitter.com/2/users/{user_id}/mentions" #Endpoint for user mentions, max 100 tweets

    #params fot the endpoint
    query_params = {'max_results': max_results,
                    'pagination_token': {}
                    }
    return (search_url, query_params)

In [66]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['pagination_token'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [41]:
def get_user_id(user_name):
    url = f'https://api.twitter.com/2/users/by/username/{user_name}'
    response = requests.request("GET", url, headers = headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()['data']['id']

In [20]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
user_id = get_user_id('McDonalds')  #McDonald's id

In [21]:
url, params = create_url(user_id)

In [23]:
tweets = connect_to_endpoint(url, headers, params)

Endpoint Response Code: 200


In [86]:
#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("../raw_data/tweets.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['tweets'])

# Inputs
max_results = 100 # Max tweets per time period
flag = True
next_token = None

# Check if flag is true
while flag:
    url, params = create_url(get_user_id('McDonalds'), max_results)
    json_response = connect_to_endpoint(url, headers, params, next_token)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
        # Save the token to use for next call
        next_token = json_response['meta']['next_token']
        if result_count is not None and result_count > 0 and next_token is not None:
            for tweet in json_response['data']:
                text = tweet['text'].replace('\n', ' ')
                csvWriter.writerow([text]) #add tweet to csv
            total_tweets += result_count
            time.sleep(5)                
    # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            for tweet in json_response['data']:
                text = tweet['text'].replace('\n', ' ')
                csvWriter.writerow([text]) #add tweet to csv
            total_tweets += result_count
            time.sleep(5)

        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
    time.sleep(5)
csvFile.close()

Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200


In [87]:
#Try adding start and end dates to get data from different months